In [1]:
import pandas as pd
import numpy as np
import os
import category_encoders as ce
import math

# visualization
import seaborn as sns
import matplotlib.pyplot as plt

## Postcode adding
I am now going to add each towns respective eircode routing key. This is the first 3 characters of an eircode. Here is an example:

***F52 NN88***

The F52 here signalling what area this address is a part of (in this case the general Boyle area)

[Reading](https://autoaddress.com/en-ie/blog/autoaddressblog/2017/06/17/it-is-surprisingly-easy-to-determine-the-eircode-routing-key-for-any-address)

Small number of towns have multiple eircodes. We will pick the one with largest percentage as per:

- Eircodes [Percentages](https://autoaddress.com/en-ie/blog/autoaddressblog/2017/06/17/it-is-surprisingly-easy-to-determine-the-eircode-routing-key-for-any-address)

We will also be using this website [here](https://thepostalcodelookup.com/?address=Ireland%2C%20Galway%2C%20Menlough%2C%20Ballinastoe%2C%20Cloonkeen%20Abbert%2C%20ireland,+ireland) to help us.

In [2]:
irl = pd.read_csv('inflated-prices-1.csv', index_col = 'Unnamed: 0')

In [3]:
irl.head()

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood,Province,Estate,Quarter,RPPI,PriceInflation
0,"14 Nuncio Road, Kilkenny, Co Kilkenny",NaN,Kilkenny,154000.0,1,1,1,1,2014,2,Kilkenny City,South East (IE),0,Nuncio Road Kilkenny,Leinster,Estate,1,82.0,279453.66
1,"25 Lintown Drive, Johnswell Road, Kilkenny",NaN,Kilkenny,70000.0,0,1,1,1,2014,1,Kilkenny City,South East (IE),0,Johnswell Road Kilkenny,Leinster,Estate,1,82.0,127024.39
2,"Archer's Mount, Castle Road, Kilkenny",NaN,Kilkenny,200000.0,1,1,1,0,2014,1,Kilkenny City,South East (IE),0,Castle Road Kilkenny,Leinster,Other,1,82.0,362926.83
3,"3 Wesley Court, William St, Kilkenny",NaN,Kilkenny,60000.0,1,1,1,1,2014,1,Kilkenny City,South East (IE),0,William Street Kilkenny,Leinster,Estate,1,82.0,108878.05
4,"4 The Parks, Weir View Castlecomer Road, Kilkenny",NaN,Kilkenny,100000.0,1,1,1,2,2014,2,Kilkenny City,South East (IE),0,Weir View Castlecomer Road Kilkenny,Leinster,Estate,1,82.0,181463.41


In [4]:
ros = irl.loc[irl['County'] == 'Roscommon']

In [5]:
roscommon_towns = list(ros['Town'].unique())
roscommon_towns

['Roscommon Town',
 'Castlerea',
 'Boyle',
 'Athlone Roscommon',
 'Tarmonbarry',
 'Elphin',
 'Ballaghaderreen Roscommon',
 'Ballinlough',
 'Ballyforan',
 'Ballyleague',
 'Ballintober',
 'Tulsk',
 'Arigna',
 'Cloonfad',
 'Croghan Roscommon',
 'Frenchpark',
 'Strokestown',
 'Rahara',
 'Ballyfarnon',
 'Creggs',
 'Carrick-On-Shannon Roscommon',
 'Lecarrow',
 'Knockcroghery',
 'Roosky Roscommon',
 'Mount Talbot',
 'Kilteevan',
 'Monksland',
 'Fuerty',
 'Curraghboy',
 'Athleague',
 'Kilglass Roscommon',
 'Loughglynn',
 'Lisacul',
 'Knockvicar',
 'Fairymount']

In [6]:
f52 = ['Boyle', 'Knockvicar', 'Keadue', 'Ballyfarnon', 'Croghan Roscommon']
n41 = ['Arigna', 'Roosky Roscommon', 'Carrick-On-Shannon Roscommon', 'Cortober']
f45 = ['Tulsk', 'Elphin', 'Frenchpark', 'Castlerea', 'Ballintober', 'Ballinlough', 'Loughglynn', 'Lisacul', 'Bellanagare'
      'Ballaghaderreen Roscommon', 'Cloonfad', 'Fairymount']
f42 = ['Strokestown', 'Knockcroghery', 'Kilteevan', 'Fuerty', 'Athleague', 'Lecarrow', 'Rahara', 'Rockfield',
      'Kilglass Roscommon', 'Ballyleague', 'Roscommon Town', 'Creggs']
n39 = ['Tarmonbarry']
h53 = ['Ballyforan', 'Taghmaconnell', 'Mount Talbot']
n37 = ['Monksland', 'Brideswell', 'Curraghboy', 'Athlone Roscommon']

In [7]:
ros.shape

(3436, 19)

In [8]:
rost = np.array(ros['Town'])
ros_post = []
for i in range(len(rost)):
    if rost[i] in f52:
        ros_post.append('F52')
    elif rost[i] in n41:
        ros_post.append('N41')
    elif rost[i] in f45:
        ros_post.append('F45')
    elif rost[i] in f42:
        ros_post.append('F42')
    elif rost[i] in n39:
        ros_post.append('N39')
    elif rost[i] in h53:
        ros_post.append('H53')
    else:
        ros_post.append('N37')

In [9]:
len(ros_post)

3436

In [10]:
ros['Eircode'] = ros_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/3176591411.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ros['Eircode'] = ros_post


In [11]:
ros.head()

,Address,Postal Code,County,Price (€),Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Year,Month,Town,Region,Apartment,Neighbourhood,Province,Estate,Quarter,RPPI,PriceInflation,Eircode
193,"Ballinturley, Four Mile House, Roscommon",NaN,Roscommon,46560.0,0,1,1,1,2014,1,Roscommon Town,West,0,Four Mile House Roscommon,Connacht,Other,1,82.0,84489.37,F42
194,"2 Hyde Court, Golf Links Rd, Roscommon",NaN,Roscommon,157000.0,1,1,1,1,2014,1,Roscommon Town,West,0,Golf Links Road Roscommon,Connacht,Estate,1,82.0,284897.56,F42
195,"32 Convent Court, Convent Rd, Roscommon",NaN,Roscommon,56000.0,1,1,1,0,2014,1,Roscommon Town,West,0,Convent Road Roscommon,Connacht,Estate,1,82.0,101619.51,F42
196,"35 Abbeyville, Galway Rd, Roscommon",NaN,Roscommon,125000.0,1,1,1,0,2014,1,Roscommon Town,West,0,Galway Road Roscommon,Connacht,Estate,1,82.0,226829.27,F42
197,"3 The Orchard, The Walk, Roscommon",NaN,Roscommon,106000.0,1,1,1,1,2014,1,Roscommon Town,West,0,The Walk Roscommon,Connacht,Estate,1,82.0,192351.22,F42


In [12]:
ros['Eircode'].isna().any()

False

### Leitrim

In [13]:
lei = irl.loc[irl['County'] == 'Leitrim']

In [14]:
leitrim_towns = list(lei['Town'].unique())
leitrim_towns

['Manorhamilton',
 'Dromahair',
 'Drumshanbo',
 'Ballinamore',
 'Tullaghan',
 'Carrick-On-Shannon Leitrim',
 'Carrigallen',
 'Mohill',
 'Kinlough',
 'Fenagh',
 'Leitrim Village',
 'Kiltyclogher',
 'Keshcarrigan',
 'Cloone',
 'Newtowngore',
 'Dromod',
 'Drumsna',
 'Dowra Leitrim',
 'Drumkeeran',
 'Bornacoola',
 'Rossinver',
 'Drumcong',
 'Glenfarne',
 'Fivemilebourne',
 'Ballinaglera',
 'Jamestown Leitrim',
 'Aghacashel',
 'Roosky Leitrim']

In [15]:
n41 = ['Carrick-On-Shannon Leitrim', 'Aghamore', 'Ballinaglera', 'Ballinamore', 'Bornacoola', 'Cloone', 'Dromod', 'Drumcong',
      'Drumkeeran', 'Drumshanbo', 'Drumsna', 'Dowra Leitrim', 'Fenagh', 'Keshcarrigan', 'Leitrim Village', 'Mohill',
       'Newtowngore', 'Rossinver', 'Tullaghan', 'Aghacashel', 'Roosky Leitrim', 'Jamestown Leitrim']
#h12 = ['Carrigallen']
f91 = ['Dromahair', 'Fivemilebourne', 'Glenfarne', 'Kiltyclogher', 'Kinlough', 'Manorhamilton']

In [16]:
leit = np.array(lei['Town'])
lei_post = []
for i in range(len(leit)):
    if leit[i] in n41:
        lei_post.append('N41')
    elif leit[i] in f91:
        lei_post.append('F91')
    else:
        lei_post.append('H12')

In [17]:
len(lei_post)

1999

In [18]:
lei.shape

(1999, 19)

In [19]:
lei['Eircode'] = lei_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/3824855931.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lei['Eircode'] = lei_post


In [20]:
lei['Eircode'].isna().any()

False

### Sligo

In [21]:
sli = irl.loc[irl['County'] == 'Sligo']

In [22]:
sli.shape

(3798, 19)

In [23]:
sligo_towns = list(sli['Town'].unique())

In [24]:
sligo_towns

['Sligo Town',
 'Ballymote',
 'Coolaney',
 'Strandhill',
 'Mullaghmore',
 'Enniscrone',
 'Rosses Point',
 'Grange Sligo',
 'Tubbercurry',
 'Ballisodare',
 'Ballintogher',
 'Riverstown Sligo',
 'Collooney',
 'Carney',
 'Monasteraden',
 'Tourlestrane',
 'Geevagh',
 'Ballinode Sligo',
 'Dromore West',
 'Cliffoney',
 'Carraroe Sligo',
 'Ballincar',
 'Aclare',
 'Gurteen',
 'Beltra',
 'Castlebaldwin',
 'Skreen',
 'Sooey',
 'Ballygawley',
 'Knocknahur',
 'Ballinafad',
 'Kilglass Sligo',
 'Bellaghy',
 'Cloonacool']

In [25]:
f91 = ['Sligo Town', 'Strandhill', 'Grange Sligo', 'Carney', 'Tubbercurry', 'Ballintogher', 'Cliffoney', 'Colloney',
      'Rosses Point', 'Mullaghmore', 'Ballisodare', 'Carraroe Sligo', 'Ballinode Sligo', 'Ballincar', 'Aclare', 'Beltra',
      'Ballygawley', 'Knocknahur', 'Cloonacool', 'Skreen', 'Tourlestrane']
f26 = ['Enniscrone', 'Dromore West', 'Kilglass Sligo']
f56 = ['Coolaney', 'Ballymote', 'Gurteen']
f45 = ['Monasteraden']
f52 = ['Riverstown Sligo', 'Castlebaldwin', 'Geevagh', 'Sooey', 'Ballinafad']
f12 = ['Bellaghy']

In [26]:
slig = np.array(sli['Town'])
sli_post = []
for i in range(len(slig)):
    if slig[i] in f52:
        sli_post.append('F52')
    elif slig[i] in f91:
        sli_post.append('F91')
    elif slig[i] in f26:
        sli_post.append('F26')
    elif slig[i] in f56:
        sli_post.append('F56')
    elif slig[i] in f45:
        sli_post.append('F45')
    else:
        sli_post.append('F12')

In [27]:
len(sli_post)

3798

In [28]:
sli['Eircode'] = sli_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/2294443118.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sli['Eircode'] = sli_post


### Mayo

In [29]:
may = irl.loc[irl['County'] == 'Mayo']

In [30]:
may.shape

(7513, 19)

In [31]:
mayo_towns = list(may['Town'].unique())
mayo_towns

['Castlebar',
 'Knock Mayo',
 'Ballina Mayo',
 'Westport',
 'Claremorris',
 'Belmullet',
 'Ballyhaunis',
 'Ballycastle',
 'Turlough',
 'Foxford',
 'Ballindine',
 'Swinford',
 'Ballinrobe',
 'Newport Mayo',
 'Kiltimagh',
 'Kilkelly',
 'Balla',
 'Charlestown',
 'Louisburgh',
 'Crossmolina',
 'Mayo',
 'Achill Sound',
 'Kilmaine',
 'Kilmeena',
 'Killala',
 'Mulranny',
 'Kilmovee',
 'Hollymount',
 'Shrule',
 'Ballaghaderreen Mayo',
 'Bohola',
 'Irishtown Mayo',
 'Cong',
 'Lahardane',
 'Finny',
 'Castlehill',
 'Bonniconlon',
 'Knockmore',
 'Cross Mayo',
 'Bangor Erris']

In [32]:
f12 = ['Knock Mayo', 'Claremorris', 'Kiltimagh', 'Swinford', 'Charlestown', 'Ballindine', 'Mayo', 'Hollymount', 'Cross Mayo',
      'Bohola']
f26 = ['Ballina Mayo', 'Foxford', 'Killala', 'Belmullet', 'Crossmolina', 'Lahardane', 'Ballycastle', 'Bangor Erris',
      'Castlehill', 'Irishtown Mayo', 'Bonniconlon', 'Knockmore']
f28 = ['Westport', 'Newport Mayo', 'Louisburgh', 'Mulranny', 'Kilmeena', 'Achill Sound']
f23 = ['Castlebar', 'Balla', 'Turlough']
f35 = ['Ballyhaunis', 'Kilkelly']
f31 = ['Ballinrobe', 'Cong', 'Kilmaine', 'Finny']
f45 = ['Kilmovee', 'Ballaghaderreen Mayo']
h91 = ['Shrule']

In [33]:
mayo_t = np.array(may['Town'])
mayo_post = []
for i in range(len(mayo_t)):
    if mayo_t[i] in f12:
        mayo_post.append('F12')
    elif mayo_t[i] in f26:
        mayo_post.append('F26')
    elif mayo_t[i] in f28:
        mayo_post.append('F28')
    elif mayo_t[i] in f23:
        mayo_post.append('F23')
    elif mayo_t[i] in f35:
        mayo_post.append('F35')
    elif mayo_t[i] in f31:
        mayo_post.append('F31')
    elif mayo_t[i] in f45:
        mayo_post.append('F45')
    else:
        mayo_post.append('H91')

In [34]:
may['Eircode'] = mayo_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/749411524.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  may['Eircode'] = mayo_post


## Galway

In [35]:
gway = irl.loc[irl['County'] == 'Galway']
gway.shape

(9740, 19)

In [36]:
galway_towns = list(gway['Town'].unique())
galway_towns

['Headford',
 'Moycullen',
 'Athenry',
 'Barna',
 'Gort',
 'Tuam',
 'Loughrea',
 'Kilcolgan',
 'Oranmore',
 'Ballinasloe',
 'Clifden',
 'Inverin',
 'Claregalway',
 'Turloughmore',
 'Clarinbridge',
 'Ballygar',
 'Knocknacarra',
 'Portumna',
 'Carraroe Galway',
 'Letterfrack',
 'Galway City',
 'Craughwell',
 'Dunmore',
 'Oughterard',
 'Kinvara',
 'Corrandulla',
 'Ardrahan',
 'Cleggan',
 'Ballyconneely',
 'Corofin Galway',
 'Williamstown',
 'Roundstone',
 'Furbo',
 'Monivea',
 'Mountbellew',
 'Woodford',
 'Glenamaddy',
 'Spiddal',
 'Clonbur',
 'Eyrecourt',
 'New Inn Galway',
 'Castlegar',
 'Ahascragh',
 'Milltown Galway',
 'Lettermore',
 'Belclare',
 'Ballymoe',
 'Peterswell',
 'Carna',
 'Menlough',
 'Ballinderreen',
 'Carnmore',
 'Camus',
 'Killimor',
 'Recess']

In [37]:
h53 = ['Ballinasloe', 'Eyrecourt', 'New Inn Galway', 'Portumna', 'Ahascragh', 'Mountbellew', 'Menlough', 'Killimor']
h91 = ['Knocknacarra', 'Oranmore', 'Moycullen', 'Galway City', 'Barna', 'Claregalway', 'Headford', 'Craughwell',
      'Oughterard', 'Ardrahan', 'Roundstone', 'Clarinbridge', 'Kinvara', 'Inverin', 'Kilcolgan', 'Corrandulla',
      'Castlegar', 'Monivea', 'Spiddal', 'Letterfrack', 'Carraroe Galway', 'Ballinderreen', 'Lettermore', 'Carna',
      'Camus', 'Carnmore', 'Peterswell', 'Furbo', 'Recess']
h65 = ['Athenry', 'Gort', 'Turloughmore']
h62 = ['Loughrea', 'Woodford']
h54 = ['Tuam', 'Corofin Galway', 'Dunmore', 'Milltown Galway', 'Belclare']
h71 = ['Clifden', 'Ballyconneely', 'Cleggan']
f45 = ['Williamstown', 'Glenamaddy', 'Ballymoe']
f12 = ['Clonbur']
f42 = ['Ballygar']

In [38]:
galway_t = np.array(gway['Town'])
galway_post = []
for i in range(len(galway_t)):
    if galway_t[i] in h53:
        galway_post.append('H53')
    elif galway_t[i] in h91:
        galway_post.append('H91')
    elif galway_t[i] in h65:
        galway_post.append('H65')
    elif galway_t[i] in h62:
        galway_post.append('H62')
    elif galway_t[i] in h54:
        galway_post.append('H54')
    elif galway_t[i] in h71:
        galway_post.append('H71')
    elif galway_t[i] in f45:
        galway_post.append('F45')
    elif galway_t[i] in f12:
        galway_post.append('F12')
    else:
        galway_post.append('F42')

In [39]:
gway['Eircode'] = galway_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/3012979648.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gway['Eircode'] = galway_post


## Cavan

In [40]:
cav = irl.loc[irl['County'] == 'Cavan']
cav.shape

(3840, 19)

In [41]:
cav_towns = list(cav['Town'].unique())
cav_towns

['Cavan Town',
 'Ballinagh',
 'Virginia',
 'Ballyjamesduff',
 'Cootehill',
 'Belturbet',
 'Mullagh Cavan',
 'Kilnaleck',
 'Ballyconnell',
 'Blacklion',
 'Stradone',
 'Bailieborough',
 'Kingscourt',
 'Bawnboy',
 'Kilcogy',
 'Mountnugent',
 'Ballyhaise',
 'Killeshandra',
 'Crossdoney',
 'Shercock',
 'Redhills',
 'Swanlinbar',
 'Butlersbridge',
 'Dowra Cavan']

In [42]:
h12 = ['Cavan Town', 'Ballinagh', 'Ballyhaise', 'Killeshandra', 'Crossdoney', 'Stradone', 'Butlersbridge']
h14 = ['Ballyconnell', 'Belturbet', 'Bawnboy', 'Redhills', 'Swanlinbar']
a82 = ['Mullagh Cavan', 'Bailieborough', 'Virginia', 'Kingscourt', 'Ballyjamesduff', 'Kilnaleck', 'Mountnugent']
h16 = ['Cootehill']
f91 = ['Blacklion']
a81 =['Shercock']
n41 = ['Dowra Cavan']
n39 = ['Kilcogy']

In [43]:
cavan_t = np.array(cav['Town'])
cavan_post = []
for i in range(len(cavan_t)):
    if cavan_t[i] in h12:
        cavan_post.append('H12')
    elif cavan_t[i] in h14:
        cavan_post.append('H14')
    elif cavan_t[i] in a82:
        cavan_post.append('A82')
    elif cavan_t[i] in h16:
        cavan_post.append('H16')
    elif cavan_t[i] in f91:
        cavan_post.append('F91')
    elif cavan_t[i] in a81:
        cavan_post.append('A81')
    elif cavan_t[i] in n41:
        cavan_post.append('N41')
    else:
        cavan_post.append('N39')

In [44]:
cav['Eircode'] = cavan_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/830075380.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cav['Eircode'] = cavan_post


## Monaghan 

In [45]:
mon = irl.loc[irl['County'] == 'Monaghan']
mon.shape

(1961, 19)

In [46]:
mon_towns = list(mon['Town'].unique())
mon_towns

['Monaghan Town',
 'Carrickmacross',
 'Castleblayney',
 'Glaslough',
 'Clones',
 'Smithborough',
 'Inniskeen',
 'Newbliss',
 'Ballybay',
 'Emyvale',
 'Scotstown',
 'Clontibret',
 'Annyalla',
 'Scotshouse',
 'Threemilehouse',
 'Ballinode Monaghan',
 'Castleshane',
 'Tydavnet',
 'Rockcorry']

In [47]:
h18 = ['Monaghan Town', 'Glaslough', 'Newbliss', 'Emyvale', 'Scotstown', 'Clontibret', 'Smithborough', 'Tydavnet',
      'Ballinode Monaghan', 'Threemilehouse', 'Rockcorry', 'Castleshane']
a81 = ['Carrickmacross']
h23 = ['Clones', 'Scotshouse']
a91 = ['Inniskeen']
a75 = ['Annyalla', 'Castleblayney', 'Ballybay']

In [48]:
monaghan_t = np.array(mon['Town'])
monaghan_post = []
for i in range(len(monaghan_t)):
    if monaghan_t[i] in h18:
        monaghan_post.append('H18')
    elif monaghan_t[i] in a75:
        monaghan_post.append('A75')
    elif monaghan_t[i] in h23:
        monaghan_post.append('H23')
    elif monaghan_t[i] in a91:
        monaghan_post.append('A91')
    else:
        monaghan_post.append('A81')

In [49]:
mon['Eircode'] = monaghan_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/3676768477.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mon['Eircode'] = monaghan_post


## Donegal

In [50]:
don = irl.loc[irl['County'] == 'Donegal']
don.shape

(7932, 19)

In [51]:
don_towns = list(don['Town'].unique())
don_towns

['Donegal Town',
 'Rathmullan',
 'Milford Donegal',
 'Buncrana',
 'Ramelton',
 'Dunfanaghy',
 'Letterkenny',
 'Kerrykeel',
 'Carndonagh',
 'Ballyshannon',
 'Dungloe',
 'Annagry',
 'Stranorlar',
 'Greencastle',
 'Ballybofey',
 'Bundoran',
 'Malin',
 'Burnfoot',
 'Convoy',
 'Lifford',
 'Killea',
 'Falcarragh',
 'Raphoe',
 'Rossnowlagh',
 'Bridgend',
 'Muff',
 'Burtonport',
 'Ballintra',
 'Kilcar',
 'Ardara',
 'Gleneely',
 'Inver',
 'Glenties',
 'Moville',
 'Dunkineely',
 'Clonmany',
 'Redcastle',
 'Culdaff',
 'Ballyliffin',
 'St Johnston',
 'Carrick',
 'Fahan',
 'Gweedore',
 'Carrigart',
 'Lettermacaward',
 'Killybegs',
 'Downings',
 'Castlefin',
 'Creeslough',
 'Kincasslagh',
 'Kilmacrennan',
 'Manorcunningham',
 'Portnablagh',
 'Killygordon',
 'Carrigans',
 'Mountcharles',
 'Glencolmcille',
 'Glen',
 'Bruckless',
 'Burt',
 'Pettigo',
 'Frosses',
 'Bunbeg',
 'Doochary',
 'Cloghan Donegal',
 'Crolly',
 'Cranford',
 'Termon',
 'Ballindrait',
 'Keadue Donegal',
 'Fintown',
 'Dunlewey']

In [52]:
f93 = ['Carndonagh', 'Buncrana', 'Castlefin', 'Lifford', 'Moville', 'Muff', 'Ballybofey', 'Carrigans', 'Fahan', 'Culdaff',
      'Killea', 'Convoy', 'Killygordon', 'Burnfoot', 'Stranorlar', 'Clonmany', 'Redcastle', 'Greencastle', 'Malin',
      'Raphoe', 'Ballyliffin', 'St Johnston', 'Gleneely', 'Burt', 'Ballindrait', 'Cloghan Donegal']
f94 = ['Glenties', 'Bundoran', 'Donegal Town', 'Glencolmcille', 'Burtonport', 'Ballyshannon', 'Glen', 'Dungloe', 'Killybegs',
      'Bridgend', 'Annagry', 'Kincasslagh', 'Portnablagh', 'Carrick', 'Lettermacaward', 'Ballintra', 'Rossnowlagh',
      'Mountcharles', 'Inver', 'Ardara', 'Bruckless', 'Kilcar', 'Dunkineely', 'Pettigo', 'Doochary', 'Frosses', 'Fintown']
f92 = ['Letterkenny', 'Rathmullan', 'Dunfanaghy', 'Carrigart', 'Ramelton', 'Kerrykeel', 'Milford Donegal', 'Manorcunningham',
      'Downings', 'Falcarragh', 'Creeslough', 'Kilmacrennan', 'Bunbeg', 'Gweedore', 'Crolly', 'Keadue Donegal', 'Termon',
      'Dunlewey', 'Cranford']

In [53]:
donegal_t = np.array(don['Town'])
donegal_post = []
for i in range(len(donegal_t)):
    if donegal_t[i] in f93:
        donegal_post.append('F93')
    elif donegal_t[i] in f94:
        donegal_post.append('f94')
    else:
        donegal_post.append('F92')

In [54]:
don['Eircode'] = donegal_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/2554596347.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  don['Eircode'] = donegal_post


## Louth

In [55]:
lu = irl.loc[irl['County'] == 'Louth']
lu.shape

(9698, 19)

In [56]:
lu_towns = list(lu['Town'].unique())
lu_towns

['Dundalk',
 'Omeath',
 'Blackrock Louth',
 'Drogheda Louth',
 'Dunleer',
 'Ardee',
 'Carlingford',
 'Baltray',
 'Greenore',
 'Castlebellingham',
 'Clogherhead',
 'Collon',
 'Termonfeckin',
 'Louth',
 'Tullyallen',
 'Tinure',
 'Tallanstown',
 'Dromiskin',
 'Annagassan',
 'Knockbridge',
 'Jenkinstown']

In [57]:
a92 = ['Drogheda Louth', 'Termonfeckin', 'Ardee', 'Clogherhead', 'Dunleer', 'Collon', 'Tullyallen', 'Baltray', 'Tinure']
a91 = ['Dundalk', 'Annagassan', 'Castlebellingham', 'Blackrock Louth', 'Carlingford', 'Omeath', 'Louth', 'Knockbridge',
      'Dromiskin', 'Tallanstown', 'Greenore', 'Jenkinstown']

In [58]:
louth_t = np.array(lu['Town'])
louth_post = []
for i in range(len(louth_t)):
    if louth_t[i] in a92:
        louth_post.append('A92')
    else:
        louth_post.append('A91')

In [59]:
lu['Eircode'] = louth_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/354774699.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lu['Eircode'] = louth_post


## Limerick

In [60]:
lim = irl.loc[irl['County'] == 'Limerick']
lim.shape

(9208, 19)

In [61]:
lim_towns = list(lim['Town'].unique())
lim_towns

['Limerick City',
 'Doon Limerick',
 'Kilmallock',
 'Athea',
 'Ballyneety',
 'Clarina',
 'Shanagolden',
 'Annacotty',
 'Raheen Limerick',
 'Castleconnell',
 'Bruff',
 'Murroe',
 'Kilteely',
 'Patrickswell',
 'Mungret',
 'Abbeyfeale',
 'Crecora',
 'Lisnagry',
 'Herbertstown',
 'Newcastle West',
 'Adare',
 'Hospital',
 'Dromcollogher',
 'Askeaton',
 'Kildimo',
 'Caherconlish',
 'Bruree',
 'Rathkeale',
 'Feohanagh',
 'Granagh',
 'Galbally',
 'Ballylanders',
 'Pallasgreen',
 'Knocklong',
 'Cappamore',
 'Grange Limerick',
 'Croagh',
 'Templeglantine',
 'Kilfinnane',
 'Croom',
 'Oola',
 'Fedamore',
 'Ballingarry Limerick',
 'Glin',
 'Ardagh Limerick',
 'Broadford Limerick',
 'Pallaskenry',
 'Foynes',
 'Tournafulla',
 'Kilmeedy',
 'Ashford Limerick',
 'Mountcollins',
 'Elton',
 'Ballyhahill',
 'Brittas Limerick',
 'Ballyorgan',
 'Strand',
 'Feenagh',
 'Ardpatrick',
 'Meanus',
 'Athlacca',
 'Kilbehenny',
 'Kilfinny',
 'Glenroe',
 'Effin',
 'Ballyagran']

In [62]:
v94 = ['Limerick City', 'Raheen Limerick', 'Rathkeale', 'Abbeyfeale', 'Mungret', 'Adare', 'Castleconnell', 'Lisnagry',
      'Annacotty', 'Croagh', 'Caherconlish', 'Pallaskenry', 'Clarina', 'Murroe', 'Ballyneety', 'Askeaton', 'Ballingarry Limerick',
      'Foynes', 'Cappamore', 'Patrickswell', 'Crecora', 'Pallasgreen', 'Kildimo', 'Kilteely', 'Athea', 'Glin', 'Shanagolden',
      'Mountcollins', 'Ballyhahill', 'Templeglantine', 'Doon Limerick', 'Kilfinny', 'Brittas Limerick']
v42 = ['Newcastle West', 'Feohanagh', 'Tournafulla', 'Kilmeedy', 'Ardagh Limerick', 'Strand', 'Ashford Limerick']
v35 = ['Bruff', 'Kilmallock', 'Croom', 'Herbertstown', 'Fedamore', 'Ballylanders', 'Knocklong', 'Hospital', 'Bruree',
      'Ardpatrick', 'Ballyorgan', 'Feenagh', 'Kilfinnane', 'Ballyagran', 'Meanus', 'Athlacca', 'Elton', 'Grange Limerick',
      'Glenroe', 'Effin', 'Granagh']
e34 = ['Oola', 'Galbally']
p56 = ['Dromcollogher', 'Broadford Limerick']
p67 = ['Kilbehenny']

In [63]:
lim_t = np.array(lim['Town'])
lim_post = []
for i in range(len(lim_t)):
    if lim_t[i] in v94:
        lim_post.append('V94')
    elif lim_t[i] in v42:
        lim_post.append('V42')
    elif lim_t[i] in v35:
        lim_post.append('V35')
    elif lim_t[i] in e34:
        lim_post.append('E34')
    elif lim_t[i] in p56:
        lim_post.append('P56')
    else:
        lim_post.append('P67')

In [64]:
lim['Eircode'] = lim_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/519894190.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lim['Eircode'] = lim_post


## Clare

In [65]:
clar = irl.loc[irl['County'] == 'Clare']
clar.shape

(6904, 19)

In [66]:
clare_towns = list(clar['Town'].unique())
clare_towns

['Ennis',
 'Lahinch',
 'Kilrush',
 'Carrigaholt',
 'Kildysart',
 'Lissycasey',
 'Clarecastle',
 'Killaloe',
 'Clonlara',
 'Ennistymon',
 'Doonbeg',
 'Meelick',
 'Kilkee',
 'Inagh',
 'Parteen',
 'Crusheen',
 'Cratloe',
 'Shannon',
 'Doolin',
 'Liscannor',
 'Bunratty',
 'Kilmihil',
 'Corofin Clare',
 'Mountshannon',
 'Bodyke',
 'Doora',
 'Ardnacrusha',
 'Tuamgraney',
 'Lisdoonvarna',
 'Inch Clare',
 'Sixmilebridge',
 'Quin',
 'Tulla',
 'Scarriff',
 'Mullagh Clare',
 'Whitegate Clare',
 'Ruan',
 'Broadford Clare',
 'Tubber Clare',
 'Ballyvaughan',
 'Kilkishen',
 'Spanish Point',
 'Kilmaley',
 'Kilfenora',
 'Milltown Malbay',
 'Cree',
 'Feakle',
 'Connolly',
 'Ballynacally',
 'Quilty',
 'Bridgetown Clare',
 'Fanore',
 'Kilnaboy',
 'Kilshanny',
 'Cross Clare',
 'Kilbaha',
 'Labasheeda']

In [67]:
v95 = ['Ennis', 'Lahinch', 'Cratloe', 'Corofin Clare', 'Liscannor', 'Clarecastle', 'Quin', 'Sixmilebridge', 'Kilkishen',
      'Milltown Malbay', 'Kilmaley', 'Bunratty', 'Kildysart', 'Mullagh Clare', 'Ennistymon', 'Doolin', 'Lisdoonvarna', 'Inagh',
      'Tulla', 'Crusheen', 'Lissycasey', 'Kilshanny', 'Quilty', 'Spanish Point', 'Ruan', 'Connolly', 'Doora', 'Ballynacally',
      'Kilnaboy', 'Kilfenora', 'Inch Clare']
v15 = ['Kilkee', 'Kilrush', 'Doonbeg', 'Carrigaholt', 'Kilmihil', 'Cree', 'Labasheeda', 'Kilbaha', 'Cross Clare']
v94 = ['Killaloe', 'Mountshannon', 'Scarriff', 'Clonlara', 'Ardnacrusha', 'Meelick', 'Parteen', 'Bodyke', 'Whitegate Clare',
      'Tuamgraney', 'Feakle', 'Bridgetown Clare', 'Broadford Clare']
v14 = ['Shannon']
h91 = ['Ballyvaughan', 'Fanore', 'Tubber Clare']

In [68]:
clar_t = np.array(clar['Town'])
clar_post = []
for i in range(len(clar_t)):
    if clar_t[i] in v95:
        clar_post.append('V95')
    elif clar_t[i] in v15:
        clar_post.append('V15')
    elif clar_t[i] in v94:
        clar_post.append('V94')
    elif clar_t[i] in h91:
        clar_post.append('H91')
    else:
        clar_post.append('V14')

In [69]:
clar['Eircode'] = clar_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/1246317912.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clar['Eircode'] = clar_post


## Tipperary

In [70]:
tipp = irl.loc[irl['County'] == 'Tipperary']
tipp.shape

(8102, 19)

In [71]:
tipp_towns = list(tipp['Town'].unique())
tipp_towns

['Clonmel',
 'Newport Tipperary',
 'Cahir',
 'Nenagh',
 'Thurles',
 'Templemore',
 'Cashel',
 'Roscrea',
 'Lorrha',
 'Borrisokane',
 'Ballina Tipperary',
 'Tipperary',
 'Clonoulty',
 'Cappawhite',
 'Dundrum Tipperary',
 'Borrisoleigh',
 'Portroe',
 'Bansha',
 'Birdhill',
 'Golden',
 'Burncourt',
 'Terryglass',
 'Fethard',
 'Clogheen',
 'Donohill',
 'Ballyporeen',
 'Emly',
 'Cullen Tipperary',
 'Cloughjordan',
 'Ardfinnan',
 'Rosegreen',
 'Mullinahone',
 'Rathcabbin',
 'Silvermines',
 'Kilsheelan',
 'Killenaule',
 'Holycross',
 'Lattin',
 'Loughmore',
 'Newcastle Tipperary',
 'Boherlahan',
 'Rearcross',
 'Dromineer',
 'Newtown Tipperary',
 'Littleton',
 'Ballycommon',
 'Hollyford',
 'Gortnahoe',
 'Riverstown Tipperary']

In [72]:
len(tipp_towns)

49

In [73]:
e45 = ['Nenagh', 'Borrisokane', 'Terryglass', 'Ballycommon', 'Portroe', 'Silvermines', 'Dromineer', 'Lorrha', 'Newtown Tipperary']
e91 = ['Clonmel', 'Fethard', 'Ardfinnan', 'Kilsheelan', 'Newcastle Tipperary']
e41 = ['Thurles', 'Templemore', 'Mullinahone', 'Killenaule', 'Borrisoleigh', 'Loughmore', 'Holycross', 'Gortnahoe',
      'Littleton']
v94 = ['Ballina Tipperary', 'Newport Tipperary', 'Birdhill', 'Rearcross']
e25 = ['Cashel', 'Golden', 'Rosegreen', 'Boherlahan', 'Clonoulty']
e53 = ['Roscrea', 'Cloughjordan']
e34 = ['Cappawhite', 'Cullen Tipperary', 'Tipperary', 'Donohill', 'Bansha', 'Emly', 'Dundrum Tipperary', 'Hollyford', 'Lattin']
e21 = ['Cahir', 'Clogheen', 'Ballyporeen', 'Burncourt']
r42 = ['Riverstown Tipperary', 'Rathcabbin']

In [74]:
tipp_t = np.array(tipp['Town'])
tipp_post = []
for i in range(len(tipp_t)):
    if tipp_t[i] in e45:
        tipp_post.append('E45')
    elif tipp_t[i] in e91:
        tipp_post.append('E91')
    elif tipp_t[i] in e41:
        tipp_post.append('E41')
    elif tipp_t[i] in v94:
        tipp_post.append('V94')
    elif tipp_t[i] in e25:
        tipp_post.append('E25')
    elif tipp_t[i] in e53:
        tipp_post.append('E53')
    elif tipp_t[i] in e34:
        tipp_post.append('E34')
    elif tipp_t[i] in e21:
        tipp_post.append('E21')
    else:
        tipp_post.append('R42')

In [75]:
tipp['Eircode'] = tipp_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/4145527976.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tipp['Eircode'] = tipp_post


## Kerry

In [76]:
ker = irl.loc[irl['County'] == 'Kerry']
ker.shape

(8776, 19)

In [77]:
ker_towns = list(ker['Town'].unique())
ker_towns

['Tralee',
 'Abbeydorney',
 'Kenmare',
 'Gneeveguilla',
 'Killarney',
 'Glenbeigh',
 'Castleisland',
 'Ballyheigue',
 'Waterville',
 'Castlemaine',
 'Ballybunion',
 'Castlegregory',
 'Listowel',
 'Dingle',
 'Ardfert',
 'Beaufort',
 'Milltown Kerry',
 'Caherdaniel',
 'Lisselton',
 'Killorglin',
 'Knocknagoshel',
 'Rathmore Kerry',
 'Sneem',
 'Annascaul',
 'Lixnaw',
 'Ballyferriter',
 'Ballinskelligs',
 'Ballylongford',
 'Tarbert',
 'Farranfore',
 'Blennerville',
 'Duagh',
 'Moyvane',
 'Asdee',
 'Castlecove',
 'Scartaglen',
 'Kilgarvan',
 'Ballyseedy',
 'Portmagee',
 'Fenit',
 'Cahersiveen',
 'Firies',
 'Lispole',
 'Kilflynn',
 'Ballyduff Kerry',
 'Dunquin',
 'Causeway',
 'Camp',
 'Ballydavid',
 'Kilmoyley',
 'Ventry',
 'Kilcummin']

In [78]:
len(ker_towns)

52

In [79]:
v93 = ['Killarney', 'Kenmare', 'Killorglin', 'Caherdaniel', 'Castlemaine', 'Castlecove', 'Beaufort', 'Milltown Kerry',
      'Glenbeigh', 'Scartaglen', 'Kilgarvan', 'Sneem', 'Farranfore', 'Firies', 'Kilcummin']
v92 = ['Lixnaw', 'Tralee', 'Ballyheigue', 'Dingle', 'Abbeydorney', 'Ardfert', 'Castleisland', 'Castlegregory', 'Annascaul',
      'Ballyseedy', 'Ballyferriter', 'Fenit', 'Knocknagoshel', 'Camp', 'Lispole', 'Causeway', 'Ballydavid', 'Kilflynn',
      'Ballyduff Kerry', 'Ventry', 'Dunquin', 'Blennerville', 'Kilmoyley']
v23 = ['Waterville', 'Ballinskelligs', 'Portmagee', 'Cahersiveen']
v31 = ['Listowel', 'Ballybunion', 'Moyvane', 'Tarbert', 'Ballylongford', 'Lisselton', 'Duagh', 'Asdee']
p51 = ['Rathmore Kerry', 'Gneeveguilla']

In [80]:
kerr_t = np.array(ker['Town'])
kerr_post = []
for i in range(len(kerr_t)):
    if kerr_t[i] in v93:
        kerr_post.append('V93')
    elif kerr_t[i] in v92:
        kerr_post.append('V92')
    elif kerr_t[i] in v23:
        kerr_post.append('V23')
    elif kerr_t[i] in v31:
        kerr_post.append('V31')
    else:
        kerr_post.append('P51')

In [81]:
ker['Eircode'] = kerr_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/1315788427.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ker['Eircode'] = kerr_post


## Waterford

In [82]:
wat = irl.loc[irl['County'] == 'Waterford']
wat.shape

(7674, 19)

In [83]:
wat_towns = list(wat['Town'].unique())
wat_towns

['Waterford City',
 'Dungarvan Waterford',
 'Dunmore East',
 'Kilmacthomas',
 'Tramore',
 'Stradbally Waterford',
 'Ardmore',
 'Passage East',
 'Lismore',
 'Cappoquin',
 'Portlaw',
 'Kill Waterford',
 'Tallow',
 'Dunhill',
 'Newtown Waterford',
 'Kilmeaden',
 'Lisduggan',
 'Rathgormack',
 'Clashmore',
 'Fenor',
 'Aglish',
 'Ballinamult',
 'Old Parish',
 'Ballymacarbry',
 'Annestown',
 'Ballytruckle',
 'Knockanore',
 'Cheekpoint',
 'Lemybrien',
 'Villierstown',
 'Kinsalebeg',
 'Glencairn']

In [84]:
x91 = ['Waterford City', 'Tramore', 'Portlaw', 'Passage East', 'Fenor', 'Dunmore East', 'Kilmeaden', 'Dunhill', 'Lisduggan',
      'Annestown', 'Kinsalebeg', 'Ballytruckle', 'Ballytruckle', 'Newtown Waterford', 'Cheekpoint']
x35 = ['Dungarvan Waterford', 'Old Parish']
p51 = ['Lismore', 'Aglish', 'Cappoquin', 'Tallow', 'Knockanore', 'Villierstown', 'Glencairn']
x42 = ['Kilmacthomas', 'Kill Waterford', 'Stradbally Waterford', 'Lemybrien']
e91 = ['Ballinamult', 'Ballymacarbry']
p36 = ['Ardmore', 'Clashmore']
e32 = ['Rathgormack']

In [85]:
wat_t = np.array(wat['Town'])
wat_post = []
for i in range(len(wat_t)):
    if wat_t[i] in x91:
        wat_post.append('X91')
    elif wat_t[i] in x35:
        wat_post.append('X35')
    elif wat_t[i] in p51:
        wat_post.append('P51')
    elif wat_t[i] in x42:
        wat_post.append('X42')
    elif wat_t[i] in e91:
        wat_post.append('E91')
    elif wat_t[i] in p36:
        wat_post.append('P36')
    else:
        wat_post.append('E32')

In [86]:
wat['Eircode'] = wat_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/2016504323.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wat['Eircode'] = wat_post


## Cork

In [142]:
cork = irl.loc[irl['County'] == 'Cork']
cork.shape

(32510, 19)

In [148]:
cork_towns = list(cork['Town'].unique())
#cork_towns

In [144]:
len(cork_towns)

127

In [145]:
t12 = ['Cork City', 'Douglas', 'Blackrock Cork', 'Bishopstown', 'Waterfall', 'Monkstown Cork', 'Togher', 'Ballygarvan',
      'Glasheen', 'Ballinhassig', 'Innishannon', 'Ballintemple', 'Tower', 'Shanbally', 'Glenbrook', 'Crossbarry']
p24 = ['Cobh']
p81 = ['Skibbereen', 'Baltimore', 'Schull', 'Glandore', 'Ballydehob', 'Leap', 'Goleen', 'Crookhaven', 'Castletownshend']
p25 = ['Midleton', 'Whitegate Cork', 'Castlemartyr', 'Cloyne', 'Ladysbridge', 'Shanagarry', 'Ballymacoda', 'Ballinacurra',
      'Dungourney']
p61 = ['Fermoy', 'Kilworth', 'Castlelyons', 'Rathcormac', 'Clondulane']
p72 = ['Bandon', 'Courtmacsherry', 'Kilbrittain', 'Timoleague', 'Ballinadee']
t45 = ['Carrigtwohill', 'Glanmire', 'Glounthaune', 'Little Island', 'Fota Island']
p67 = ['Mitchelstown']

p17 = ['Kinsale', 'Belgooly', 'Minane Bridge', 'Nohoval', 'Ballinspittle']
p36 = ['Youghal', 'Killeagh']
p31 = ['Ballincollig', 'Ovens', 'Aherla']
p43 = ['Carrigaline', 'Crosshaven', 'Ringaskiddy', 'Riverstick', 'Fountainstown']
p51 = ['Mallow', 'Banteer', 'Kanturk', 'Doneraile', 'Newmarket', 'Liscarroll', 'Buttevant', 'Castletownroche', 'Conna',
      'Glanworth', 'Lombardstown', 'Meelin', 'Bweeng', 'Shanballymore', 'Rockchapel', 'Dromahane', 'Churchtown Cork',
      'Ballydesmond', 'Lismire']

p85 = ['Clonakilty', 'Rosscarbery', 'Ardfield', 'Lyre']
p12 = ['Macroom', 'Dripsey', 'Ballingeary', 'Ballymakeera', 'Millstreet', 'Coachford', 'Aghabullogue', 'Carrigadrohid',
      'Ballyvourney', 'Coolea']
p47 = ['Drimoleague', 'Dunmanway', 'Drinagh', 'Ballineen']
p56 = ['Charleville', 'Milford Cork']

p75 = ['Bantry', 'Castletownbere', 'Union Hall', 'Durrus', 'Allihies', 'Adrigole', 'Kealkill', 'Kilcrohane', 'Ballylickey']
t23 = ['Blarney', 'Cloghroe', 'Glengarriff', 'Montenotte', 'Mayfield', 'Farran']
t34 = ['Whitechurch']
t56 = ['Knockraha', 'Watergrasshill']
p32 = ['Donoughmore']
p14 = ['Crookstown Cork', 'Cloughduv']

In [149]:
cork_t = np.array(cork['Town'])
cork_post = []
for i in range(len(cork_t)):
    if cork_t[i] in t12:
        cork_post.append('T12')
    elif cork_t[i] in p24:
        cork_post.append('P24')
    elif cork_t[i] in p81:
        cork_post.append('P81')
    elif cork_t[i] in p25:
        cork_post.append('P25')
    elif cork_t[i] in p61:
        cork_post.append('P61')
    elif cork_t[i] in p72:
        cork_post.append('P72')
    elif cork_t[i] in t45:
        cork_post.append('T45')
    elif cork_t[i] in p67:
        cork_post.append('P67')
    elif cork_t[i] in p17:
        cork_post.append('P17')
    elif cork_t[i] in p36:
        cork_post.append('P36')
    elif cork_t[i] in p31:
        cork_post.append('P31')
    elif cork_t[i] in p43:
        cork_post.append('P43')
    elif cork_t[i] in p51:
        cork_post.append('P51')
    elif cork_t[i] in p85:
        cork_post.append('P85')
    elif cork_t[i] in p12:
        cork_post.append('P12')
    elif cork_t[i] in p47:
        cork_post.append('P47')
    elif cork_t[i] in p56:
        cork_post.append('P56')
    elif cork_t[i] in p75:
        cork_post.append('P75')
    elif cork_t[i] in t23:
        cork_post.append('T23')
    elif cork_t[i] in t34:
        cork_post.append('T34')
    elif cork_t[i] in t56:
        cork_post.append('T56')
    elif cork_t[i] in p32:
        cork_post.append('P32')
    else:
        cork_post.append('P14')

In [150]:
cork['Eircode'] = cork_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/1574716197.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cork['Eircode'] = cork_post


## Longford

In [87]:
lon = irl.loc[irl['County'] == 'Longford']
lon.shape

(2093, 19)

In [88]:
lon_towns = list(lon['Town'].unique())
lon_towns

['Longford Town',
 'Edgeworthstown',
 'Ballymahon',
 'Legan',
 'Moydow',
 'Granard',
 'Aughnacliffe',
 'Newtownforbes',
 'Ballinalee',
 'Newtowncashel',
 'Drumlish',
 'Abbeylara',
 'Killashee',
 'Ballinamuck',
 'Abbeyshrule',
 'Lisryan',
 'Colehill',
 'Killoe',
 'Ardagh Longford',
 'Lanesborough',
 'Keenagh']

In [89]:
n39 = ['Longford Town', 'Ballymahon', 'Edgeworthstown', 'Newtownforbes', 'Drumlish', 'Killashee', 'Granard', 'Newtowncashel',
      'Ballinalee', 'Aughnacliffe', 'Legan', 'Lisryan', 'Colehill', 'Abbeylara', 'Killoe', 'Abbeyshrule', 'Ardagh Longford',
      'Lanesborough', 'Ballinamuck', 'Moydow', 'Keenagh']

In [90]:
lon_t = np.array(lon['Town'])
lon_post = []
for i in range(len(lon_t)):
    lon_post.append('N39')

In [91]:
lon['Eircode'] = lon_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/348769471.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lon['Eircode'] = lon_post


## Westmeath

In [92]:
wmeath = irl.loc[irl['County'] == 'Westmeath']
wmeath.shape

(6416, 19)

In [93]:
wmeath_towns = list(wmeath['Town'].unique())
wmeath_towns

['Mullingar',
 'Athlone Westmeath',
 'Moate',
 'Castlepollard',
 'Kilbeggan',
 'Ballynacargy',
 'Rochfortbridge',
 'Delvin',
 'Multyfarnham',
 'Kinnegad',
 'Mount Temple',
 'Tyrrellspass',
 'Raharney',
 'Castletown Geoghegan',
 'Collinstown',
 'Clonmellon',
 'Ballymore Westmeath',
 'Rathowen',
 'Milltownpass',
 'Rathconrath',
 'Ballinea',
 'Streamstown',
 'Drumraney',
 'Coole',
 'Tang',
 'Rosemount Westmeath']

In [94]:
n37 = ['Athlone Westmeath', 'Moate', 'Mount Temple', 'Drumraney', 'Rosemount Westmeath']
n91 = ['Mullingar', 'Castlepollard', 'Kilbeggan', 'Ballymore Westmeath', 'Kinnegad', 'Rathowen', 'Rochfortbridge',
      'Multyfarnham', 'Collinstown', 'Tyrrellspass', 'Delvin', 'Streamstown', 'Ballinea', 'Ballynacargy', 'Raharney',
      'Coole', 'Milltownpass', 'Rathconrath', 'Castletown Geoghegan']
c15 = ['Clonmellon']
n39 = ['Tang']

In [95]:
wmeath_t = np.array(wmeath['Town'])
wmeath_post = []
for i in range(len(wmeath_t)):
    if wmeath_t[i] in n37:
        wmeath_post.append('N37')
    elif wmeath_t[i] in n91:
        wmeath_post.append('N91')
    elif wmeath_t[i] in c15:
        wmeath_post.append('C15')
    else:
        wmeath_post.append('N39')

In [96]:
wmeath['Eircode'] = wmeath_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/1712840505.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wmeath['Eircode'] = wmeath_post


## Offaly

In [97]:
off = irl.loc[irl['County'] == 'Offaly']
off.shape

(3932, 19)

In [98]:
off_towns = list(off['Town'].unique())
off_towns

['Tullamore',
 'Edenderry',
 'Birr',
 'Banagher',
 'Portarlington Offaly',
 'Shinrone',
 'Clara Offaly',
 'Kinnitty',
 'Moneygall',
 'Daingean',
 'Geashill',
 'Kilcormac',
 'Rhode',
 'Ferbane',
 'Ballycumber',
 'Killeigh',
 'Bracknagh',
 'Cloghan Offaly',
 'Ballinagar',
 'Belmont',
 'Walsh Island',
 'Mucklagh',
 'Rahan',
 'Horseleap Offaly',
 'Clonygowan',
 'Shannonbridge']

In [99]:
r42 = ['Birr', 'Banagher', 'Shinrone', 'Belmont', 'Ferbane', 'Kilcormac', 'Cloghan Offaly', 'Kinnitty']
r32 = ['Portarlington Offaly']
r35 = ['Tullamore', 'Killeigh', 'Clara Offaly', 'Daingean', 'Ballinagar', 'Geashill', 'Rhode', 'Ballycumber', 'Walsh Island',
      'Mucklagh', 'Clonygowan', 'Rahan']
r45 = ['Edenderry']
r51 = ['Bracknagh']
e53 = ['Moneygall']
n37 = ['Shannonbridge', 'Horseleap Offaly']

In [100]:
off_t = np.array(off['Town'])
off_post = []
for i in range(len(off_t)):
    if off_t[i] in n37:
        off_post.append('N37')
    elif off_t[i] in r42:
        off_post.append('R42')
    elif off_t[i] in r32:
        off_post.append('R32')
    elif off_t[i] in r35:
        off_post.append('R35')
    elif off_t[i] in r45:
        off_post.append('R45')
    elif off_t[i] in r51:
        off_post.append('R51')
    else:
        off_post.append('E53')

In [101]:
off['Eircode'] = off_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/2565711037.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  off['Eircode'] = off_post


## Carlow

In [102]:
car = irl.loc[irl['County'] == 'Carlow']
car.shape

(2622, 19)

In [103]:
car_towns = list(car['Town'].unique())
car_towns

['Carlow Town',
 'Tullow',
 'Hacketstown',
 'Ballinabranna',
 'Ballon',
 'Borris',
 'Myshall',
 'Ardattin',
 'Kildavin',
 'Leighlinbridge',
 'Rathvilly',
 'Clonegal',
 'Muine Bheag',
 'Nurney Carlow',
 'Palatine',
 'St Mullins',
 'Tinryland',
 'Graiguenamanagh Carlow',
 'Old Leighlin',
 'Bilboa',
 'Ballymurphy']

In [104]:
r93 = ['Tullow', 'Graiguenamanagh Carlow', 'Carlow Town', 'Ballon', 'Palatine', 'Nurney Carlow', 'Leighlinbridge', 'Hacketstown',
      'Rathvilly', 'Old Leighlin', 'Ballinabranna', 'Ardattin', 'Tinryland', 'Bilboa']
y21 = ['Clonegal', 'Kildavin']
r95 = ['Borris', 'St Mullins', 'Ballymurphy']
r21 = ['Myshall', 'Muine Bheag']

In [105]:
car_t = np.array(car['Town'])
car_post = []
for i in range(len(car_t)):
    if car_t[i] in r93:
        car_post.append('R93')
    elif car_t[i] in y21:
        car_post.append('Y21')
    elif car_t[i] in r95:
        car_post.append('R95')
    else:
        car_post.append('R21')

In [106]:
car['Eircode'] = car_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/1541190376.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  car['Eircode'] = car_post


## Laois

In [107]:
laois = irl.loc[irl['County'] == 'Laois']
laois.shape

(4752, 19)

In [108]:
laois_towns = list(laois['Town'].unique())
laois_towns

['Portlaoise',
 'Portarlington Laois',
 'Killenard',
 'Mountmellick',
 'Rathdowney',
 'Stradbally Laois',
 'Mountrath',
 'Abbeyleix',
 'Durrow Laois',
 'Ballinakill',
 'Ballyroan',
 'Ballacolla',
 'Castletown Laois',
 'Ballybrittas',
 'Emo',
 'Ballickmoyler',
 'Ballylinan',
 'Clonaslee',
 'Crettyard',
 'Timahoe',
 'Killeshin',
 'Rosenallis',
 'The Swan',
 'Vicarstown',
 'Clough',
 'Ballyhide',
 'Kilbricken',
 'Errill',
 'Newtown Laois',
 'Ballybrophy',
 'Spink']

In [109]:
r32 = ['Portlaoise', 'Stradbally Laois', 'Mountmellick', 'Rathdowney', 'Mountrath', 'Ballybrittas', 'Killenard',
      'Portarlington Laois', 'Durrow Laois', 'Ballacolla', 'Ballinakill', 'Abbeyleix', 'Emo', 'Clonaslee', 'Ballyroan',
      'Timahoe', 'Castletown Laois', 'Newtown Laois', 'Rosenallis', 'Spink', 'Vicarstown', 'Errill', 'Ballybrophy',
      'Kilbricken', 'Clough']
r14 = ['Ballylinan', 'The Swan']
r93 = ['Crettyard', 'Ballickmoyler', 'Killeshin', 'Ballyhide']

In [110]:
laois_t = np.array(laois['Town'])
laois_post = []
for i in range(len(laois_t)):
    if laois_t[i] in r32:
        laois_post.append('R32')
    elif laois_t[i] in r14:
        laois_post.append('R14')
    else:
        laois_post.append('R93')

In [111]:
laois['Eircode'] = laois_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/3455821781.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laois['Eircode'] = laois_post


## Kilkenny

In [112]:
kilk = irl.loc[irl['County'] == 'Kilkenny']
kilk.shape

(4037, 19)

In [113]:
kilk_towns = list(kilk['Town'].unique())
kilk_towns

['Kilkenny City',
 'Ferrybank Kilkenny',
 'Thomastown',
 'Kilmacow',
 'Kilmoganny',
 'Gowran',
 'Piltown',
 'Graiguenamanagh Kilkenny',
 'Callan',
 'Mooncoin',
 'Castlecomer',
 'Paulstown',
 'Goresbridge',
 'Urlingford',
 'Stoneyford',
 'Bennettsbridge',
 'Inistioge',
 'Freshford',
 'Ballyragget',
 'Fiddown',
 'Mullinavat',
 'Slieverue',
 'Ballyhale',
 'Johnstown Kilkenny',
 'Dunnamaggin',
 'Windgap',
 'Tullaroan',
 'Hugginstown',
 'Glenmore',
 'Kells Kilkenny',
 'Galmoy',
 'Knocktopher',
 'Tullogher',
 'Carrigeen',
 'The Rower',
 'Johnswell',
 'Clogh',
 'Castlewarren',
 'Dungarvan Kilkenny',
 'Moneenroe']

In [114]:
r95 = ['Castlecomer', 'Ballyragget', 'Inistioge', 'Callan', 'Knocktopher', 'Gowran', 'Goresbridge', 'Paulstown',
      'Thomastown', 'Castlewarren', 'Kells Kilkenny', 'Kilkenny', 'Kilkenny City', 'Graiguenamanagh Kilkenny', 'Bennettsbridge',
      'Freshford', 'Stoneyford', 'The Rower', 'Dunnamaggin', 'Tullaroan', 'Hugginstown', 'Johnswell', 'Ballyhale',
      'Kilmoganny', 'Dungarvan Kilkenny', 'Moneenroe', 'Windgap', 'Clogh']
e32 = ['Fiddown', 'Piltown']
x91 = ['Ferrybank Kilkenny', 'Mullinavat', 'Mooncoin', 'Carrigeen', 'Kilmacow', 'Slieverue', 'Tullogher']
e41 = ['Johnstown Kilkenny', 'Urlingford', 'Galmoy']
y34 = ['Glenmore']

In [115]:
kilk_t = np.array(kilk['Town'])
kilk_post = []
for i in range(len(kilk_t)):
    if kilk_t[i] in r95:
        kilk_post.append('R95')
    elif kilk_t[i] in e32:
        kilk_post.append('E32')
    elif kilk_t[i] in x91:
        kilk_post.append('X91')
    elif kilk_t[i] in e41:
        kilk_post.append('E41')
    else:
        kilk_post.append('Y34')

In [116]:
kilk['Eircode'] = kilk_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/445531358.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kilk['Eircode'] = kilk_post


## Meath

In [117]:
mea = irl.loc[irl['County'] == 'Meath']
mea.shape

(14064, 19)

In [118]:
mea_towns = list(mea['Town'].unique())
mea_towns

['Navan',
 'Ashbourne',
 'Ratoath',
 'Dunshaughlin',
 'Dunboyne',
 'Summerhill',
 'Drogheda Meath',
 'Trim',
 'Athboy',
 'Laytown',
 'Kells Meath',
 'Oldcastle',
 'Donacarney',
 'Julianstown',
 'Bettystown',
 'Enfield',
 'Ballivor',
 'Longwood',
 'Mornington',
 'Stamullen',
 'Duleek',
 'Slane',
 'Drumree',
 'Clonee',
 'Ardcath',
 'Donore',
 'Kilmessan',
 'Nobber',
 'Clonard',
 'Batterstown',
 'Carlanstown',
 'Kildalkey',
 'Drumconrath',
 'Kilcloon',
 'Kentstown',
 'Rathmolyon',
 'Robinstown',
 'Gormanston',
 'Carnaross',
 'Bellewstown',
 'Moynalty',
 'Bective']

In [119]:
a85 = ['Dunshaughlin', 'Ratoath', 'Kilcloon', 'Drumree']
c15 = ['Navan', 'Trim', 'Athboy', 'Ballivor', 'Slane', 'Kildalkey', 'Kilmessan', 'Kentstown', 'Bective', 'Drumconrath',
      'Robinstown']
a82 = ['Kells Meath', 'Donore', 'Nobber', 'Oldcastle', 'Carnaross', 'Carlanstown', 'Moynalty']
a86 = ['Betterstown', 'Dunboyne']
a84 = ['Ashbourne']
k32 = ['Stamullen', 'Gormanston']
a92 = ['Drogheda Meath', 'Bettystown', 'Mornington', 'Laytown', 'Duleek', 'Julianstown', 'Donacarney', 'Bellewstown']
a83 = ['Enfield', 'Longwood', 'Summerhill', 'Clonard', 'Rathmolyon']
d15 = ['Clonee']
a42 = ['Ardcath']

In [120]:
meath_t = np.array(mea['Town'])
meath_post = []
for i in range(len(meath_t)):
    if meath_t[i] in a85:
        meath_post.append('A85')
    elif meath_t[i] in c15:
        meath_post.append('C15')
    elif meath_t[i] in a82:
        meath_post.append('A82')
    elif meath_t[i] in a86:
        meath_post.append('A86')
    elif meath_t[i] in a84:
        meath_post.append('A84')
    elif meath_t[i] in k32:
        meath_post.append('K32')
    elif meath_t[i] in a92:
        meath_post.append('A92')
    elif meath_t[i] in a83:
        meath_post.append('A83')
    elif meath_t[i] in d15:
        meath_post.append('D15')
    else:
        meath_post.append('D42')

In [121]:
mea['Eircode'] = meath_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/2902502723.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mea['Eircode'] = meath_post


## Kildare

In [122]:
kil = irl.loc[irl['County'] == 'Kildare']
kil.shape

(17708, 19)

In [123]:
kild_towns = list(kil['Town'].unique())
kild_towns

['Rathangan',
 'Naas',
 'Celbridge',
 'Newbridge Kildare',
 'Straffan',
 'Clane',
 'Leixlip',
 'Kilcullen',
 'Maynooth',
 'Calverstown',
 'Athy',
 'Sallins',
 'Castledermot',
 'Kilcock',
 'Coill Dubh',
 'Kill Kildare',
 'Derrinturn',
 'Monasterevin',
 'Athgarvan',
 'Nurney Kildare',
 'Kildare',
 'Carbury Kildare',
 'Ballymore Eustace',
 'Prosperous',
 'Johnstown Kildare',
 'Kildangan',
 'Allenwood',
 'Robertstown',
 'Rathcoffey',
 'Crookstown Kildare',
 'Kilkea',
 'Johnstownbridge',
 'Ballitore',
 'Suncroft',
 'Moone',
 'Narraghmore',
 'Caragh',
 'Allen',
 'Staplestown',
 'Timolin',
 'Ardclough',
 'Brownstown',
 'Kilteel']

In [124]:
w23 = ['Maynooth', 'Straffan', 'Celbridge', 'Leixlip', 'Kilcock', 'Ardclough']
r14 = ['Athy', 'Timolin', 'Ballitore', 'Castledermot', 'Crookstown Kildare', 'Kilkea', 'Narraghmore', 'Moone']
w12 = ['Newbridge Kildare']
w91 = ['Clane', 'Naas', 'Caragh', 'Kill Kildare', 'Johnstown Kildare', 'Sallins', 'Allenwood', 'Carbury', 'Ballymore Eustace',
      'Derrinturn', 'Prosperous', 'Coill Dubh', 'Rathcoffey', 'Robertstown', 'Allen', 'Kilteel', 'Staplestown']
r56 = ['Kilcullen', 'Athgarvan', 'Suncroft', 'Calverstown', 'Brownstown']
r51 = ['Rathangan', 'Kildare', 'Kildare Village', 'Nurney Kildare']
w34 = ['Monasterevin', 'Kildangan']
a83 = ['Johnstownbridge']

In [125]:
kild_t = np.array(kil['Town'])
kild_post = []
for i in range(len(kild_t)):
    if kild_t[i] in w23:
        kild_post.append('W23')
    elif kild_t[i] in r14:
        kild_post.append('R14')
    elif kild_t[i] in w12:
        kild_post.append('W12')
    elif kild_t[i] in w91:
        kild_post.append('W91')
    elif kild_t[i] in r56:
        kild_post.append('R56')
    elif kild_t[i] in r51:
        kild_post.append('R51')
    elif kild_t[i] in a92:
        kild_post.append('W34')
    else:
        kild_post.append('A83')

In [126]:
kil['Eircode'] = kild_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/2849659743.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kil['Eircode'] = kild_post


## Wexford

In [127]:
wex = irl.loc[irl['County'] == 'Wexford']
wex.shape

(10337, 19)

In [128]:
wex_towns = list(wex['Town'].unique())
wex_towns

['Gorey',
 'Enniscorthy',
 'Murrintown',
 'Saltmills',
 'Ballindaggin',
 'Kilmuckridge',
 'New Ross',
 'Courtown',
 'Kilmore Quay',
 'Kilrane',
 'Castlebridge',
 'Rosslare Strand',
 'Curracloe',
 'Taghmon',
 'Duncormick',
 'Bridgetown Wexford',
 'Wellingtonbridge',
 'Oylegate',
 'Blackwater',
 'Ballyhack',
 'Riverchapel',
 'Barntown',
 'Ferns',
 'Wexford',
 'Kilmore Wexford',
 'Tagoat',
 'Campile',
 'Bunclody',
 'Arthurstown',
 'Rosslare Harbour',
 'Adamstown Wexford',
 'Cleariestown',
 'Ballymurn',
 'Inch Wexford',
 'Bannow',
 'Ballygarrett',
 'Newbawn',
 'Camolin',
 'Oulart',
 'Clonroche',
 'Clohamon',
 'Crossabeg',
 'Duncannon',
 'Ballycanew',
 'Killurin',
 'Bree',
 'Ballycullane',
 'Broadway',
 'Rosbercon Wexford',
 'Craanford',
 'Hollyfort',
 'The Ballagh',
 'Tacumshane',
 'Kiltealy',
 'Castletown Wexford',
 'Morriscastle',
 'Rathnure',
 'Ballywilliam',
 'Coolgreany',
 'Ballyhogue',
 'Monageer']

In [129]:
len(wex_towns)

61

In [133]:
y25 = ['Gorey', 'Kilmuckridge', 'Courtown', 'Riverchapel', 'Inch Wexford', 'Ballygarrett', 'Oulart', 'Ballycanew',
      'Craanford', 'Hollyfort', 'Castletown Wexford', 'Morriscastle', 'Coolgreany']
y21 = ['Enniscorthy', 'Ballindaggin', 'Curracloe', 'Oylegate', 'Blackwater', 'Ferns', 'Bunclody', 'Adamstown Wexford',
      'Ballymurn', 'Camolin', 'Clonroche', 'Clohamon', 'Killurin', 'Bree', 'The Ballagh', 'Kiltealy', 'Rathnure',
      'Ballywilliam', 'Ballyhogue', 'Monageer']
y35 = ['Murrintown', 'Kilmore Quay', 'Kilrane', 'Castlebridge', 'Rosslare Strand', 'Taghmon', 'Duncormick', 'Bridgetown Wexford',
      'Wellingtonbridge', 'Barntown', 'Wexford', 'Wexford Town', 'Kilmore Wexford', 'Tagoat', 'Rosslare Harbour', 'Cleariestown',
      'Bannow', 'Newbawn', 'Crossabeg', 'Broadway', 'Tacumshane']
y34 = ['Saltmills', 'New Ross', 'Ballyhack', 'Campile', 'Arthurstown', 'Duncannon', 'Ballycullane', 'Rosbercon Wexford']

In [134]:
wex_t = np.array(wex['Town'])
wex_post = []
for i in range(len(wex_t)):
    if wex_t[i] in y25:
        wex_post.append('Y25')
    elif wex_t[i] in y21:
        wex_post.append('Y21')
    elif wex_t[i] in y35:
        wex_post.append('Y35')
    else:
        wex_post.append('Y34')

In [135]:
wex['Eircode'] = wex_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/142115142.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wex['Eircode'] = wex_post


## Wicklow

In [130]:
wick = irl.loc[irl['County'] == 'Wicklow']
wick.shape

(10642, 19)

In [131]:
wick_towns = list(wick['Town'].unique())
wick_towns

['Arklow',
 'Kilquade',
 'Greystones',
 'Delgany',
 'Blessington',
 'Bray',
 'Rathnew',
 'Aughrim Wicklow',
 'Ashford Wicklow',
 'Redcross',
 'Glenealy',
 'Kilcoole',
 'Dunlavin',
 'Kilmacanogue',
 'Rathdrum',
 'Newtownmountkennedy',
 'Shillelagh',
 'Avoca',
 'Roundwood',
 'Hollywood',
 'Wicklow',
 'Enniskerry',
 'Tinahely',
 'Kilpedder',
 'Grangecon',
 'Carnew',
 'Newcastle Wicklow',
 'Donard',
 'Baltinglass',
 'Barndarrig',
 'Kiltegan',
 'Woodenbridge',
 'Laragh',
 'Kilbride',
 'Knockananna',
 'Valleymount',
 'Glendalough',
 'Ballinaclash',
 'Coolboy',
 'Annamoe',
 'Killincarrig']

In [132]:
len(wick_towns)

41

In [136]:
y14 = ['Arklow', 'Aughrim Wicklow', 'Shillelagh', 'Avoca', 'Tinahely', 'Carnew', 'Woodenbridge', 'Knockananna', 'Coolboy']
a63 = ['Kilquade', 'Greystones', 'Delgany', 'Kilcoole', 'Newtownmountkennedy', 'Kilpedder', 'Newcastle Wicklow', 'Killincarrig']
w91 = ['Blessington', 'Dunlavin', 'Hollywood', 'Grangecon', 'Donard', 'Baltinglass', 'Kiltegan', 'Valleymount']
a98 = ['Bray', 'Kilmacanogue', 'Roundwood', 'Enniskerry', 'Laragh', 'Glendalough', 'Annamoe']
a67 = ['Rathnew', 'Ashford Wicklow', 'Redcross', 'Glenealy', 'Rathdrum', 'Wicklow', 'Wicklow Town', 'Kilbride', 'Ballinaclash']

In [137]:
wick_t = np.array(wick['Town'])
wick_post = []
for i in range(len(wick_t)):
    if wick_t[i] in y14:
        wick_post.append('Y14')
    elif wick_t[i] in a63:
        wick_post.append('A63')
    elif wick_t[i] in w91:
        wick_post.append('W91')
    elif wick_t[i] in a98:
        wick_post.append('A98')
    else:
        wick_post.append('A67')

In [138]:
wick['Eircode'] = wick_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/2861942926.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wick['Eircode'] = wick_post


## Dublin

In [139]:
dub = irl.loc[irl['County'] == 'Dublin']
dub.shape

(52777, 19)

In [147]:
dub_towns = list(dub['Town'].unique())
#dub_towns

In [141]:
len(dub_towns)

136

In [146]:
d6w = ['Terenure', 'Templeogue', 'Kimmage']
a94 = ['Blackrock Dublin', 'Mount Merrion', 'Stillorgan', 'Monkstown Dublin', 'Booterstown', 'Deansgrange', 'Merrion']
d24 = ['Saggart', 'Rathcoole', 'Citywest', 'Tallaght', 'Brittas Dublin', 'Ballyboden']
k67 = ['Swords']
d16 = ['Dundrum', 'Sandyford', 'Ballinteer', 'Rathfarnham', 'Knocklyon', 'Firhouse']
d12 = ['Walkinstown', 'Crumlin', 'Drimnagh', 'Bluebell', 'Kilnamanagh']
k34 = ['Skerries', 'Loughshinny']

d18 = ['Stepaside', 'Carrickmines', 'Foxrock', 'Ballybrack', 'Shankill', 'Leopardstown', 'Glencullen', 'Rathmichael',
      'Kilternan']
d14 = ['Milltown Dublin', 'Churchtown Dublin', 'Clonskeagh', 'Kilmacud', 'Goatstown']
k32 = ['Balbriggan', 'Balrothery', 'Naul']

a42 = ['Adamstown Dublin', 'Garristown']
k78 = ['Lucan']
d04 = ['Donnybrook', 'Ballsbridge', 'Sandymount', 'Ringsend', 'Irishtown Dublin']
d13 = ['Portmarnock', 'Howth', 'Clongriffin', 'Sutton', 'Baldoyle', 'Donaghmede', 'Kilbarrack', 'Balgriffin', 'Bayside']
k56 = ['Rush']
k36 = ['Malahide', 'Donabate', 'Portrane']
d08 = ['Rialto', 'Inchicore', 'Portobello', 'Kilmainham', 'The Coombe', 'Islandbridge']
d07 = ['Stoneybatter', 'Cabra', 'Smithfield', 'Phibsborough', 'Ashtown', 'Grangegorman', 'Broadstone']
d20 = ['Palmerstown', 'Chapelizod']

d06 = ['Rathgar', 'Rathmines', 'Dartry', 'Harolds Cross', 'Ranelagh']
d09 = ['Drumcondra', 'Santry', 'Whitehall', 'Beaumont']
a96 = ['Dalkey', 'Loughlinstown', 'Killiney', 'Cabinteely', 'Sandycove', 'Glenageary', 'Glasthule', 'Sallynoggin',
      'Dún Laoghaire', 'Cherrywood']
d05 = ['Coolock', 'Killester', 'Raheny', 'Donnycarney', 'Artane']
d10 = ['Ballyfermot', 'Cherry Orchard']
d15 = ['Castleknock', 'Hollystown', 'Clonsilla', 'Blanchardstown', 'Ongar', 'Mulhuddart', 'Tyrrelstown']
d22 = ['Clondalkin', 'Newcastle Dublin', 'Ballymount']
d11 = ['Finglas', 'Glasnevin', 'Ballymun', 'Poppintree']
d03 = ['Fairview', 'East Wall', 'Clontarf', 'Marino']

a41 = ['Ballyboughal']
k45 = ['Lusk',]
d17 = ['Kinsealy', 'Darndale', 'Clare Hall']
d01 = ['North Strand', 'Ballybough', 'North Wall']
a45 = ['Oldtown']

In [151]:
dub_t = np.array(dub['Town'])
dub_post = []
for i in range(len(dub_t)):
    if dub_t[i] in d6w:
        dub_post.append('D6W')
    elif dub_t[i] in a94:
        dub_post.append('A94')
    elif dub_t[i] in d24:
        dub_post.append('D24')
    elif dub_t[i] in k67:
        dub_post.append('K67')
    elif dub_t[i] in d16:
        dub_post.append('D16')
    elif dub_t[i] in d12:
        dub_post.append('D12')
    elif dub_t[i] in k34:
        dub_post.append('K34')
    elif dub_t[i] in d18:
        dub_post.append('D18')
    elif dub_t[i] in d14:
        dub_post.append('D14')
    elif dub_t[i] in k32:
        dub_post.append('K32')
    elif dub_t[i] in a42:
        dub_post.append('A42')
    elif dub_t[i] in k78:
        dub_post.append('K78')
    elif dub_t[i] in d04:
        dub_post.append('D04')
    elif dub_t[i] in d13:
        dub_post.append('D13')
    elif dub_t[i] in k56:
        dub_post.append('K56')
    elif dub_t[i] in k36:
        dub_post.append('K36')
    elif dub_t[i] in d08:
        dub_post.append('D08')
    elif dub_t[i] in d07:
        dub_post.append('D07')
    elif dub_t[i] in d20:
        dub_post.append('D20')
    elif dub_t[i] in d06:
        dub_post.append('D06')
    elif dub_t[i] in d09:
        dub_post.append('D09')
    elif dub_t[i] in a96:
        dub_post.append('A96')
    elif dub_t[i] in d05:
        dub_post.append('D05')
    elif dub_t[i] in d10:
        dub_post.append('D10')
    elif dub_t[i] in d15:
        dub_post.append('D15')
    elif dub_t[i] in d22:
        dub_post.append('D22')
    elif dub_t[i] in d11:
        dub_post.append('D11')
    elif dub_t[i] in d03:
        dub_post.append('D03')
    elif dub_t[i] in a41:
        dub_post.append('A41')
    elif dub_t[i] in k45:
        dub_post.append('K45')
    elif dub_t[i] in d17:
        dub_post.append('D17')
    elif dub_t[i] in d01:
        dub_post.append('D01')
    else:
        dub_post.append('A45')

In [152]:
dub['Eircode'] = dub_post

C:\Users\user\AppData\Local\Temp/ipykernel_17884/623330143.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dub['Eircode'] = dub_post


### Join it all together...

In [159]:
lll = pd.concat([ros, lei, sli, may, gway, cav, mon, don, lu, lim, clar, tipp, ker, wat, cork,
                lon, wmeath, off, car, laois, kilk, mea, kil, wex, wick, dub], ignore_index=True)

In [166]:
lll.head()

,Address,County,Price (€),Not Full Market Price,VAT Exclusive,Property Size Description,Year,Month,Town,Region,Neighbourhood,PriceInflation,Eircode
0,"Ballinturley, Four Mile House, Roscommon",Roscommon,46560.0,0,1,1,2014,1,Roscommon Town,West,Four Mile House Roscommon,84489.37,F42
1,"2 Hyde Court, Golf Links Rd, Roscommon",Roscommon,157000.0,1,1,1,2014,1,Roscommon Town,West,Golf Links Road Roscommon,284897.56,F42
2,"32 Convent Court, Convent Rd, Roscommon",Roscommon,56000.0,1,1,0,2014,1,Roscommon Town,West,Convent Road Roscommon,101619.51,F42
3,"35 Abbeyville, Galway Rd, Roscommon",Roscommon,125000.0,1,1,0,2014,1,Roscommon Town,West,Galway Road Roscommon,226829.27,F42
4,"3 The Orchard, The Walk, Roscommon",Roscommon,106000.0,1,1,1,2014,1,Roscommon Town,West,The Walk Roscommon,192351.22,F42


In [161]:
#lll.to_csv('eircodes-all-cols.csv')

In [162]:
lll.drop(columns=['RPPI', 'Estate', 'Apartment', 'Province', 'Quarter', 'Postal Code', 'Description of Property'],
        inplace = True)

In [165]:
lll.head()

,Address,County,Price (€),Not Full Market Price,VAT Exclusive,Property Size Description,Year,Month,Town,Region,Neighbourhood,PriceInflation,Eircode
0,"Ballinturley, Four Mile House, Roscommon",Roscommon,46560.0,0,1,1,2014,1,Roscommon Town,West,Four Mile House Roscommon,84489.37,F42
1,"2 Hyde Court, Golf Links Rd, Roscommon",Roscommon,157000.0,1,1,1,2014,1,Roscommon Town,West,Golf Links Road Roscommon,284897.56,F42
2,"32 Convent Court, Convent Rd, Roscommon",Roscommon,56000.0,1,1,0,2014,1,Roscommon Town,West,Convent Road Roscommon,101619.51,F42
3,"35 Abbeyville, Galway Rd, Roscommon",Roscommon,125000.0,1,1,0,2014,1,Roscommon Town,West,Galway Road Roscommon,226829.27,F42
4,"3 The Orchard, The Walk, Roscommon",Roscommon,106000.0,1,1,1,2014,1,Roscommon Town,West,The Walk Roscommon,192351.22,F42


In [164]:
#lll.to_csv('less-columns-eircodes.csv')